# NLP con Long-Short Term Memory (LSTM)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion2/2-nlp-with-lstm.ipynb)

En este notebook implementaremos un clasificador de noticias en español utilizando la arquitectura de red LSTM. La idea es tener un punto de referencia para comparar cuando observemos la parte de transformers, por lo que utilizaremos el mismo dataset y tarea de ejemplo. Utilizarémos las utilidades de tokenización de huggingface transformers para ayudarnos con esta tarea.

#### Referencias
- Dataset: https://huggingface.co/datasets/MarcOrfilaCarreras/spanish-news
- [Long Short-Term Memory](https://www.researchgate.net/publication/13853244_Long_Short-Term_Memory#fullTextFileContent)

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

/tmp/ipykernel_34267/2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt && pip install -r requirements.txt

### Cargando el dataset
Este es un dataset pequeño de articulos de noticias en idioma español con sus respectivas categorías. El dataset está disponible en el HuggingFace Hub y puede ser fácilmente descargado con la librería.

In [3]:
from datasets import load_dataset
import warnings
import os

warnings.filterwarnings("ignore")
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
dataset = load_dataset('MarcOrfilaCarreras/spanish-news', split='train')
dataset

Generating train split: 100%|██████████| 10200/10200 [00:00<00:00, 23599.51 examples/s]


Dataset({
    features: ['language', 'category', 'newspaper', 'hash', 'text'],
    num_rows: 10200
})

Observemos uno de sus registros

In [4]:
dataset[1]

{'language': 'es',
 'category': 'play',
 'newspaper': 'de_lector_a_lector',
 'hash': 'b387bc0a5ad68524c8aa5da489555ca41d5a3575',
 'text': 'El coraje de ser, de Mónica Cavallé, la aventura del autoconocimiento filosófico.Todos experimentamos momentos de plenitud vinculados a la expresión directa y auténtica de nosotros mismos: momentos de contemplación de la belleza del mundo en que nuestros sentidos se abren como si lo vieran por primera vez, de intimidad y comunión con otro ser humano, de fluidez creativa, de expresión confiada y libre… Estos momentos permiten intuir lo que puede ser una vida en la que no meramente se existe, sino en la que se vive en todo el sentido de esta palabra.Esta vida solo es posible cuando sabemos quiénes somos, cuando nos conocemos a nosotros mismos de modo experiencial: no cuando nos llenamos de ideas sobre nosotros, sino cuando nos asentamos en nuestro ser real, más allá de nuestras defensas, máscaras y falsos yoes.El coraje de ser es una invitación a aden

Para los efectos de esta tarea, nos servirán el texto y la categoría naturalmente.

A manera general, observemos que tan largos o cortos tienden a ser los textos.

In [5]:
text_lengths = [len(row['text']) for row in dataset]
print(f"Texto más corto: {min(text_lengths)}")
print(f"Texto más largo: {max(text_lengths)}")
print(f"Longitud promedio: {sum(text_lengths) / len(text_lengths)}")

Texto más corto: 501
Texto más largo: 204324
Longitud promedio: 4218.154509803921


Estos valores son la cantidad de *caractéres* que tiene las secuencias. Una decisión ingenua pero útil en este momento podría ser ajustar la longitud de las secuencias que vamos a usar para el entrenamiento a unos 2000 tokens. Esto podría ser suficiente para capturar una porción significativa de los textos.

## Definiendo el Tokenizer

Ahora, vamos a definir el tokenizer para nuestra tarea. Para ahorrarnos tiempo, vamos a entrenar uno basado en gpt2, pero ajustandolo a nuestro dataset. Para ello, debemos seleccionar una muestra representativa de nuestro dataset, como no es muy grande, casi que podemos usarlo todo. Luego, debemos definir el tamaño del vocabulario, es decir, cuantos tokens únicos queremos soportar en nuestro tokenizador. Para que un modelo de lenguaje funcione moderadamente bien para una tarea de clasificación, considerando el tamaño de nuestro corpus, deberíamos definir unos 50 mil tokens.

In [6]:
from tqdm.auto import tqdm
from transformers import AutoTokenizer
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode


length = 10000
iter_dataset = iter(dataset)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

byte_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k, v in byte_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

def batch_iterator(batch_size: int = 10):
    for _ in tqdm(range(0, length, batch_size)):
        yield [next(iter_dataset)['text'] for _ in range(batch_size)]

spanish_news_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=50000, initial_alphabet=base_vocab)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
100%|██████████| 1000/1000 [00:06<00:00, 159.29it/s]


Exploremos ahora el tokenizador obtenido.

In [7]:
tokens = sorted(spanish_news_tokenizer.vocab.items(), key=lambda x: x[1], reverse=False)
print(f"Vocabulario: {spanish_news_tokenizer.vocab_size} tokens")
print("Primeros 15 tokens:")
print([f"{spanish_news_tokenizer.convert_tokens_to_string([t])}" for t, _ in tokens[:15]])
print("15 tokens de en medio:")
print([f"{spanish_news_tokenizer.convert_tokens_to_string([t])}" for t, _ in tokens[1000:1015]])
print("Últimos 15 tokens:")
print([f"{spanish_news_tokenizer.convert_tokens_to_string([t])}" for t, _ in tokens[-15:]])

Vocabulario: 50000 tokens
Primeros 15 tokens:
['<|endoftext|>', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.']
15 tokens de en medio:
['pul', 'ece', ' Car', ' op', ' hech', 'ome', ' Ar', ' cen', 'ensa', 'ismo', 'ós', 'ord', 'at', '”.', 'lec']
Últimos 15 tokens:
['yang', 'zidina', 'zambique', '\x96', ' ud', ' �', 'deado', ' each', ' eches', ' eternidad', ' eferentes', ' eBook', ' deple', ' dedu', ' deduce']


Vemos que los primeros tokens corresponden a caracteres especiales y puntiación. Luego en el medio tenemos una combinación entre palabras completas y cortadas, el tokenizador se encarga de encontrar las frecuencias más comunes y asi partir las palabras por aquellas partes que tienden a repetirse mas. Esto es muy útil para trabajar con modelos de lenguaje ya que el modelo se vuelve robusto a diferentes ramificaciones de palabras e incluso a errores de tipografía. Finalmente, al final, vemos que tenemos más palabras cortadas y palabras muy especiales. Algo importante aquí es que podamos ver que los tokens tienen sentido con respecto a nuestro corpus.

Ahora veamos como convierte el tokenizador una oración muy sencilla:

In [8]:
spanish_news_tokenizer.pad_token = '[PAD]'
spanish_news_tokenizer("hola mundo!", max_length=8, truncation=True, padding='max_length')

{'input_ids': [72, 1086, 1039, 1, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0]}

Lo que obtenemos de vuelta son los ids de cada token según el vocabulario. Ahora algo importante que notamos aquí es el *padding*, durante el entrenamiento, queremos que las secuencias sean de tamaño fijo, para asi operar comodamente con matrices. Pero ya vimos que no todos los textos tienen la misma longitud. Entonces que hacer? para los que son más largos que una longitud dada simplemente cortamos, pero para los que son más cortos, debemos *rellenar* lo faltante con un *token especial de relleno o padding*. Y es justo lo que definimos allí, cuando la cadena es inferior a 8 **tokens**, entonces debemos hacer padding hasta que se cumplan los 8.

Ahora, notemos que "hola mundo!" son 2 palabras, 9 letras, 1 espacio y 1 simbolo para un total de 11 caracteres, pero vemos que el resultado son 4 tokens y el padding. Esto es trabajo del tokenizador. Cuando lo entrenamos con nuestro corpus, el tokenizador computó las frecuencias de palabras y sus partes, tal como vimos arriba, entonces, estos tokens juntos forman la frase original, observemos:

In [9]:
spanish_news_tokenizer("hola mundo!", max_length=8, truncation=True, padding='max_length').tokens()

['h', 'ola', 'Ġmundo', '!', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

Claramente vemos los 4 tokens como cadenas independientes.

### Definiendo el dataset de pytorch
Ahora podemos proceder a definir el dataset. Esto debería ser muy sencillo dado que nuestro dataset es pequeño y ya tenemos el tokenizador listo.

In [10]:
import torch
import numpy as np
from typing import Tuple, Dict
from torch.utils.data import Dataset

class SpanishNewsDataset(Dataset):

    def __init__(self, tokenizer, dataset, seq_length: int = 512):
        self.tokenizer = tokenizer
        self.tokenizer.pad_token = '[PAD]'
        self.dataset = dataset
        self.seq_length = seq_length
        # Definimos estos dos mapas para facilitarnos la tarea
        # de traducir de nombres de categoría a ids de categoría.
        self.id_2_class_map = dict(enumerate(np.unique(dataset[:]['category'])))
        self.class_2_id_map = {v: k for k, v in self.id_2_class_map.items()}
        self.num_classes = len(self.id_2_class_map)

    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        text, y = self.dataset[index]['text'], self.dataset[index]['category']
        y = self.class_2_id_map[y]
        data = {k: torch.tensor(v) for k, v in self.tokenizer(text, max_length=self.seq_length, truncation=True, padding='max_length').items()}
        data['y'] = torch.tensor(y)
        return data


    def __len__(self):
        return len(self.dataset)

Ahora instanciaremos el dataset entero. Para este experimento, definiremos un tamaño máximo de secuencia de 2048 **tokens**. Que según nuestra intuición arriba, debería ser suficiente para la tarea.

In [11]:
max_len = 512 
spanish_news_dataset = SpanishNewsDataset(spanish_news_tokenizer, dataset, seq_length=max_len)
assert len(spanish_news_dataset) == len(dataset)

Y luego, procedemos a hacer el train-val-test split y crear los dataloaders.

In [12]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader

batch_size = 4 if not IN_COLAB else 12
train_dataset, val_dataset, test_dataset = random_split(spanish_news_dataset, lengths=[0.8, 0.1, 0.1])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)

## Definición del modelo LSTM

Ahora vamos a configurar un módulo pytorch simple para este problema. Vamos ha utilizar los embeddings, que vendrían siendo los vectores de palabra. Pytorch nos ofrece una capa con la que directamente podemos entrenarlos a partir de los token ids obtenidos. El resto consistirá en invocar una capa LSTM seguida de una capa densa para la clasificación.

Recordemos que las redes recurrentes como las LSTM por diseño enlazan todas las dimensiones del vector de entrada, formando así la secuencia, la estructura natural que necesitamos representar.

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMBlock(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
    
    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, _) = self.lstm(embedded)
        return hidden[-1]


### Definición del clasificador

Finalmente, definimos el modelo en si. Este modelo constará de 3 capas:

- La tokenización, tal como la definimos anteriormente.
- El bloque LSTM, que acabamos de decinir.
- Una capa densa adicional que servirá como clasificador de aquello que nos entregue la capa del transformer.

Como este es un LightningModule, aquí definiremos el resto de funciones utilitarias para el entrenamiento de la tarea.

In [28]:
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchmetrics import Accuracy

class SpanishNewsClassifierWithLSTM(LightningModule):

    def __init__(self, vocab_size: int, num_classes: int, emb_dim: int, hidden_dim: int = 128):
        super(SpanishNewsClassifierWithLSTM, self).__init__()
        self.num_classes = num_classes
        self.lstm = LSTMBlock(vocab_size, emb_dim, hidden_dim, num_classes)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
            nn.LogSoftmax(dim=1)
        )

        self.train_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.val_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.test_acc = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        embeddings = self.lstm(x)
        return self.classifier(embeddings)

    
    def training_step(self, batch):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.train_acc(y_hat, y)
        self.log('train-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train-acc', self.train_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def validation_step(self, batch):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.val_acc(y_hat, y)
        self.log('val-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val-acc', self.val_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def test_step(self, batch):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        self.test_acc(y_hat, y)
        self.log('test-acc', self.test_acc, prog_bar=True, on_step=False, on_epoch=True)


    def predict_step(self, batch):
        x = batch['input_ids']
        return self(x)


    def configure_optimizers(self):
        optimizer =  torch.optim.AdamW(self.parameters(), lr=1e-3, weight_decay=1e-5)
        return optimizer

    
model = SpanishNewsClassifierWithLSTM(vocab_size=spanish_news_tokenizer.vocab_size, num_classes=spanish_news_dataset.num_classes, emb_dim=256)

tb_logger = TensorBoardLogger('tb_logs', name='LSTMClassifier')
callbacks=[EarlyStopping(monitor='train-loss', patience=3, mode='min')]
trainer = Trainer(max_epochs=10, devices=1, logger=tb_logger, callbacks=callbacks, precision="16-mixed")

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Using bfloat16 Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | lstm       | LSTMBlock          | 13.1 M | train
1 | classifier | Sequential         | 200 K  | train
2 | train_acc  | MulticlassAccuracy | 0      | train
3 | val_acc    | MulticlassAccuracy | 0      | train
4 | test_acc   | MulticlassAccuracy | 0      | train
----------------------------------------------------------
13.3 M    Trainable params
0         Non-trainable params
13.3 M    Total params
53.321    Total estimated model params size (MB)
16        Modules in train mode
0         Modules in eval mode


Epoch 9: 100%|██████████| 2040/2040 [02:31<00:00, 13.50it/s, v_num=1, val-loss=2.340, val-acc=0.158, train-loss=2.280, train-acc=0.181]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 2040/2040 [02:31<00:00, 13.47it/s, v_num=1, val-loss=2.340, val-acc=0.158, train-loss=2.280, train-acc=0.181]


Observemos el proceso de entrenamiento

In [29]:
%load_ext tensorboard

In [30]:
%tensorboard --logdir tb_logs/

Y como es de esperarse, realizaremos la validación contra el conjunto de prueba.

In [31]:
model.eval()
trainer.test(model, test_loader)

Testing DataLoader 0: 100%|██████████| 255/255 [00:02<00:00, 119.12it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test-acc          │    0.15980392694473267    │
└───────────────────────────┴───────────────────────────┘

[{'test-acc': 0.15980392694473267}]

### Haciendo predicciones

Finalmente, vamos a hacer uso del modelo y ver que tan bueno es para la clasificación de noticias.

In [32]:
predictions = trainer.predict(model, test_loader)
predictions = torch.cat(predictions, dim=0)
predictions = torch.argmax(predictions, dim=-1)
predictions = [spanish_news_dataset.id_2_class_map[pred] for pred in predictions.numpy()]

Predicting DataLoader 0: 100%|██████████| 255/255 [00:01<00:00, 130.29it/s]


In [33]:
import pandas as pd

test_indices = test_dataset.indices
df = pd.DataFrame(data={
    "texto": dataset[test_indices]['text'],
    "tokens": [spanish_news_tokenizer(v)['input_ids'] for v in dataset[test_indices]['text']],
    "categoría": dataset[test_indices]['category'],
    'predicción': predictions
}, index=test_indices)

df['tokens_string'] = df.tokens.apply(lambda t: spanish_news_tokenizer.convert_ids_to_tokens(t))
df = df[["texto", "tokens", "tokens_string", "categoría", "predicción"]]
df.style.set_table_styles(
    [
        {'selector': 'td', 'props': [('word-wrap', 'break-word')]}
    ]
)
df.head(15)

Token indices sequence length is longer than the specified maximum sequence length for this model (1709 > 1024). Running this sequence through the model will result in indexing errors


,texto,tokens,tokens_string,categoría,predicción
1629,TAG Heuer acaba de anunciar una curiosa y pecu...,"[27693, 46981, 3747, 259, 11280, 347, 17325, 2...","[TAG, ĠHeuer, Ġacaba, Ġde, Ġanunciar, Ġuna, Ġc...",tech,military
4186,Decía la mitología que el Ave Fénix es aquel q...,"[36, 4961, 280, 20555, 288, 289, 348, 710, 456...","[D, ecÃŃa, Ġla, ĠmitologÃŃa, Ġque, Ġel, ĠA, ve...",sport,military
5676,"Un año más, el concurso que elige a la mejor c...","[2183, 795, 383, 12, 289, 9091, 288, 22358, 26...","[Un, ĠaÃ±o, ĠmÃ¡s, ,, Ġel, Ġconcurso, Ġque, Ġe...",alimentation,play
1538,"El pasado mes de febrero, el inversor George S...","[544, 1145, 946, 259, 1937, 12, 289, 23175, 96...","[El, Ġpasado, Ġmes, Ġde, Ġfebrero, ,, Ġel, Ġin...",tech,medicine
9541,"Repartirá un dividendo complementario de 0,09 ...","[4716, 493, 44812, 297, 14285, 28343, 259, 165...","[Re, par, tirÃ¡, Ġun, Ġdividendo, Ġcomplementa...",economy,medicine
8633,Volvo sigue siendo noticia en los últimos días...,"[42631, 1944, 1397, 5268, 279, 313, 1664, 1253...","[Volvo, Ġsigue, Ġsiendo, Ġnoticia, Ġen, Ġlos, ...",motor,play
9722,Cada cuatro años el calendario se ve alterado ...,"[10183, 1552, 640, 289, 6394, 309, 885, 34585,...","[Cada, Ġcuatro, ĠaÃ±os, Ġel, Ġcalendario, Ġse,...",economy,politics
6129,"Aníbal Tortoriello, diputado nacional de Junto...","[2541, 295, 5937, 3868, 403, 3971, 378, 12, 84...","[An, ÃŃ, bal, ĠTor, tor, iel, lo, ,, Ġdiputado...",politics,play
3576,Las lesiones son una parte fundamental en el d...,"[1492, 4361, 558, 347, 769, 3250, 279, 289, 59...","[Las, Ġlesiones, Ġson, Ġuna, Ġparte, Ġfundamen...",sport,medicine
338,Era cuestión de tiempo que 'Barbie' siguiera r...,"[17838, 3850, 259, 903, 288, 750, 11492, 7, 44...","[Era, ĠcuestiÃ³n, Ġde, Ġtiempo, Ġque, Ġ', Barb...",play,play


In [34]:
errors = df[df['categoría'] != df['predicción']]
errors.head(15)

,texto,tokens,tokens_string,categoría,predicción
1629,TAG Heuer acaba de anunciar una curiosa y pecu...,"[27693, 46981, 3747, 259, 11280, 347, 17325, 2...","[TAG, ĠHeuer, Ġacaba, Ġde, Ġanunciar, Ġuna, Ġc...",tech,military
4186,Decía la mitología que el Ave Fénix es aquel q...,"[36, 4961, 280, 20555, 288, 289, 348, 710, 456...","[D, ecÃŃa, Ġla, ĠmitologÃŃa, Ġque, Ġel, ĠA, ve...",sport,military
5676,"Un año más, el concurso que elige a la mejor c...","[2183, 795, 383, 12, 289, 9091, 288, 22358, 26...","[Un, ĠaÃ±o, ĠmÃ¡s, ,, Ġel, Ġconcurso, Ġque, Ġe...",alimentation,play
1538,"El pasado mes de febrero, el inversor George S...","[544, 1145, 946, 259, 1937, 12, 289, 23175, 96...","[El, Ġpasado, Ġmes, Ġde, Ġfebrero, ,, Ġel, Ġin...",tech,medicine
9541,"Repartirá un dividendo complementario de 0,09 ...","[4716, 493, 44812, 297, 14285, 28343, 259, 165...","[Re, par, tirÃ¡, Ġun, Ġdividendo, Ġcomplementa...",economy,medicine
8633,Volvo sigue siendo noticia en los últimos días...,"[42631, 1944, 1397, 5268, 279, 313, 1664, 1253...","[Volvo, Ġsigue, Ġsiendo, Ġnoticia, Ġen, Ġlos, ...",motor,play
9722,Cada cuatro años el calendario se ve alterado ...,"[10183, 1552, 640, 289, 6394, 309, 885, 34585,...","[Cada, Ġcuatro, ĠaÃ±os, Ġel, Ġcalendario, Ġse,...",economy,politics
6129,"Aníbal Tortoriello, diputado nacional de Junto...","[2541, 295, 5937, 3868, 403, 3971, 378, 12, 84...","[An, ÃŃ, bal, ĠTor, tor, iel, lo, ,, Ġdiputado...",politics,play
3576,Las lesiones son una parte fundamental en el d...,"[1492, 4361, 558, 347, 769, 3250, 279, 289, 59...","[Las, Ġlesiones, Ġson, Ġuna, Ġparte, Ġfundamen...",sport,medicine
2814,"Fuente de la imagen, APEl atún azul constituye...","[3942, 259, 280, 2026, 12, 8978, 544, 13395, 3...","[Fuente, Ġde, Ġla, Ġimagen, ,, ĠAP, El, ĠatÃºn...",astronomy,play


## Conclusiones

- En este caso tenemos una tarea de clasificación de texto de múltiples clases.
- Estamos usando un bloque LSTM como featurizer, es decir lo usamos para extraer features de las secuencias de entrada con las cuales harémos predicciones luego.
- Nótese que de las capas LSTM, solo nos interesa la última, ya que esta recupera todas las operaciones enalazadas anteriores.
- Observamos que el modelo toma su tiempo en entrenar, esto es natural debido al diseño de las LSTM, donde por cada paso de tiempo se debe computar un gradiente, por lo que el computo es mucho mayor.
- Los resultados de clasificación no son malos, pero tampoco son excelentes. Podemos hacerlo mejor?